# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import multilabel_confusion_matrix, classification_report, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn import svm

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///clean_f8.db')
df = pd.read_sql_table('clean_f8_2', con=engine)

X = df['message']
Y = df[df.columns[4:]]

In [34]:
df.shape

(26216, 40)

In [30]:
pd.set_option('display.max_colwidth', None)
df[df['aid_related'] == 1][['message']].head(20)

,message
1,Is the Hurricane over or is it not over
3,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
7,"Please, we need tents and water. We are in Silo, Thank you!"
9,I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )
10,"There's nothing to eat and water, we starving and thirsty."
12,"I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks"
13,"Let's do it together, need food in Delma 75, in didine area"
15,"A Comitee in Delmas 19, Rue ( street ) Janvier, Impasse Charite #2. We have about 500 people in a temporary shelter and we are in dire need of Water, Food, Medications, Tents and Clothes. Please stop by and see us."
16,We need food and water in Klecin 12. We are dying of hunger. Impasse Chretien Klecin 12 extended ( extension ) We are hungry and sick.
21,"I'm in Laplaine, I am a victim"


In [33]:
df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']].mean()

related                   0.766478
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

In [36]:
df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']].sum(axis=1).value_counts()

0     6122
1     5309
3     3609
5     2956
4     2276
6     1903
7     1347
8      880
9      577
2      443
10     344
11     183
12     107
13      57
14      38
15      20
17      12
16       8
18       7
19       5
20       4
21       4
22       2
27       2
24       1
dtype: int64

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [41]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
#        ('clf', MultiOutputClassifier(RandomForestClassifier())),
        ('clf', MultiOutputClassifier(svm.SVC()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000027DEC579280>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
Y_pred = pipeline.predict(X_test)

In [13]:
categories = Y.columns.values
for i, cat in enumerate(categories):
    print(cat)
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.73      0.27      0.39      1465
           1       0.82      0.97      0.89      5089

    accuracy                           0.81      6554
   macro avg       0.77      0.62      0.64      6554
weighted avg       0.80      0.81      0.78      6554

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5412
           1       0.89      0.44      0.59      1142

    accuracy                           0.89      6554
   macro avg       0.89      0.72      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6014
           1       0.90      0.29      0.44       540

    accuracy                           0.94      6554
   macro avg       0.92      0.64      0.70      6554
weighted avg       0.94      0.94      0.92      6554

storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5943
           1       0.79      0.38      0.51       611

    accuracy                           0.93      6554
   macro avg       0.87      0.68      0.74      6554
weighted avg       0.93      0.93      0.92      6554

fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6490
           1       0.00      0.00      0.00        64

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

earthqua

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [42]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000027DEC579280>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=SVC()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000027DEC579280>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=SVC()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__estima

In [27]:
params_grid = [{
                'tfidf__smooth_idf': (True, False),
                'clf': [MultiOutputClassifier(RandomForestClassifier())],
                'clf__n_estimators': [100, 200],
                },
                {
                'tfidf__smooth_idf': (True, False),                    
                'clf': [MultiOutputClassifier(svm.SVC())],
                'clf__C': [0.1, 1, 10, 100, 1000],
                'clf__gamma': [1, 0.1, 0.01, 0.001, 0.0001],                
                }]

cv = GridSearchCV(pipeline, param_grid=params_grid, scoring=recall_score, verbose=3, n_jobs=6) #TODO add up to 8 more threads

In [43]:
parameters = {
    'clf':  [MultiOutputClassifier(svm.SVC())],
    'clf__estimator__C': [0.1, 1, 10, 100, 1000],
    'clf__estimator__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [44]:
cv.fit(X_train, Y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=0.1;, score=nan total time= 3.0min
[CV 2/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=0.1;, score=nan total time= 2.9min
[CV 3/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=0.1;, score=nan total time= 3.0min
[CV 4/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=0.1;, score=nan total time= 3.0min
[CV 5/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=0.1;, score=nan total time= 2.9min
[CV 1/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=1;, score=nan total time= 4.9min
[CV 2/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=1;, score=nan total time= 4.9min
[CV 3/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__estimator__C=1;, score=nan total time= 5.0min
[CV 4/5] END clf=MultiOutputClassifier(estimator=SVC()), clf__esti

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multioutput.py", line 434, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "C:\Prog

ValueError: The number of classes has to be greater than one; got 1 class

In [47]:
labels

array([0, 1], dtype=int64)

In [45]:
Y_pred = cv.predict(X_test)

NotFittedError: This MultiOutputClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
    labels = np.unique(Y_pred)
    confusion_mat = multilabel_confusion_matrix(Y_test, Y_pred, labels=labels)
    accuracy = (Y_pred == Y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Model Accuracy:", accuracy.mean())
    print("Accuracy Breakdown:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

Labels: [0 1]
Confusion Matrix:
 [[[ 393 1072]
  [ 149 4940]]

 [[5351   61]
  [ 638  504]]]
Model Accuracy: 0.9458600345844776
Accuracy Breakdown: related                   0.813702
request                   0.893348
offer                     0.994812
aid_related               0.768081
medical_help              0.923253
medical_products          0.952395
search_and_rescue         0.974519
security                  0.981996
military                  0.963686
child_alone               1.000000
water                     0.944614
food                      0.925999
shelter                   0.930119
clothing                  0.983827
money                     0.977113
missing_people            0.988709
refugees                  0.967043
death                     0.955905
other_aid                 0.875343
infrastructure_related    0.939274
transport                 0.957736
buildings                 0.951938
electricity               0.982911
tools                     0.993134
hospitals   

NameError: name 'cv' is not defined

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
Y_true

NameError: name 'Y_true' is not defined

In [13]:
categories = Y.columns.values
for i, cat in enumerate(categories):
    print(cat)
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]), zero_division=0)


related
              precision    recall  f1-score   support

           0       0.76      0.27      0.40      1523
           1       0.82      0.97      0.89      5031

    accuracy                           0.81      6554
   macro avg       0.79      0.62      0.64      6554
weighted avg       0.80      0.81      0.77      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5452
           1       0.86      0.44      0.58      1102

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5838
           1       0.85      0.40      0.54       716

    accuracy                           0.93      6554
   macro avg       0.89      0.69      0.75      6554
weighted avg       0.92      0.93      0.91      6554

shelter
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5973
           1       0.82      0.23      0.36       581

    accuracy                           0.93      6554
   macro avg       0.87      0.61      0.66      6554
weighted avg       0.92      0.93      0.91      6554

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6463
           1       1.00      0.09      0.16        91

    accuracy                           0.99      6554
   macro avg       0.99      0.54      0.58      6554
weighted avg       0.99      0.99      0.98      6554

mo

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6512
           1       0.00      0.00      0.00        42

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.00      0.00      0.00        75

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

aid

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.